In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes
%matplotlib inline
plt.ioff()

In [39]:
from glob import glob
from datetime import datetime
from itertools import *
from operator import itemgetter
import json

In [3]:
spec_files = glob('profiling/*/specs')
spec_files.sort()
specs = [np.genfromtxt(f, delimiter=' ', dtype=int)
         for f in spec_files]

In [4]:
linestyles = ["-","--","-.",":"]
colors = ('blue','black','red','orange','green','magenta','yellow', 'cyan', 'purple', 'firebrick')
colorcycle = cycle(colors)
lastpe = False
line_def = []
color_def = []
for spec in specs:
    pe, comp, impl, mode = (spec[2], spec[4], spec[5], spec[6])
    if not lastpe or lastpe != pe:
        color = next(colorcycle)
        linecycle = cycle(linestyles)
    line  = next(linecycle)
    lastpe = pe
    line_def.append(line)
    color_def.append(color)

In [5]:
def set_keys(*indices):
    """Returns a function that returns a tuple of key values"""
    def get_keys(seq, indices=indices):
        keys = []
        for i in indices:
            keys.append(seq[i])
        return tuple(keys)
    return get_keys

In [6]:
def get_average_times(dat, dirCol, stepCol, timeCol):
    dat.sort(order=['dir', 'step'])
    result = {}
    for (c_dir, c_step), rows in groupby(dat, key=set_keys('dir','step')):
        #print c_dir, c_step, rows
        c_dir, c_step, total_time, counter = None, None, 0, 0
        for row in rows:
            if not c_dir:
                c_dir = row[dirCol]
            if not c_step:
                c_step = row[stepCol]
            total_time+=row[timeCol]
            counter += 1
        result.setdefault(c_dir, []).append([c_step, total_time*1.0/counter])
    result2 = []
    for c_dir, rows in result.iteritems():
        inner = []
        for row in rows:
            inner.append(row[1])
        result2.append(inner)
    return np.array([np.array(rows) for rows in result2])

In [7]:
def is_axis(a):
    return not type(a) is np.ndarray

Header = ['timestamp', 'dir', 'step', 'ilp-gen']

In [8]:
def read_results(prefix, name, dtype, data_column, since):
    f = 'profiling/{0}-{1}-results.csv'.format(prefix, name)
    def mkdate(text):
        return datetime.strptime(text, '%Y-%m-%dT%H:%M:%S.%f')
    def mkstep(text):
        return int(text[0:2])
    dat = np.genfromtxt(f, delimiter=',', names=True, dtype=dtype, converters={'timestamp':mkdate, 'step':mkstep})
    if since:
        dat = dat[dat['timestamp'] > since ]
    dat2 = get_average_times(dat, 1, 2, data_column).transpose()
    print 'Loaded {0} records for {1} ({2[0]}x{2[1]} unique) ~= {3} run(s)'.format(
        len(dat), name, dat2.shape, len(dat)/dat2.size)
    return dat2

# Need testing, if still working with change kinds

In [9]:
def read_gen_results(impl, since = None):
    return read_results('gen', impl, ('datetime64[us]', int, int, float), 3, since)
def read_sol_results(solver, since = None):
    return read_result('sol', solver, ('datetime64[us]', int, int, int, int, int, float, float), 7, since)
def read_att_result(name = 'profiling/att-percentages.csv'):
    def cdir(text):
        prefix = text[:-4]
        number = int(text[-3:])
        if prefix == 'update':
            return number
        elif prefix == 'sw':
            return 31+number
        elif prefix == 'res':
            return 35+number
        elif prefix == '':
            pass
        else:
            print 'Unknown prefix "{0}"'.format(prefix)
    dat = np.genfromtxt(name, delimiter=',', names=True,
                        #dtype=(int, float, float, float, float, float, float, float),
                        converters={'dir': cdir})
    dat.sort(axis=0)
    print 'Loaded {0} attribute metrics'.format(len(dat))
    return dat

In [10]:
def safe(a, i, start = 0):
    try:
        return a[start:,i]
    except IndexError:
        return np.zeros(a[:,0].size)

## ILP-Re-Generation and Solving Time

- ILP-Re-Generation describes the time it takes for the Scheme implementations to evaluate the attribute computing the ILP
    - it does not include the time to read from and write to disk
    - "Java" denotes the Java- and EMF-based generation
- ILP Solving describes the time it takes the solvers (GLPK and Gurobi in this case) to solve the generated ILP
    - "GLPK (Java)" denotes the time to solve the ILP generated by the Java-based generation with GLPK. Its format was modifiert slightly to be accepted by GLPK, as it was originally generated for lp_solve
    - "GLPK (Scheme)" denotes the time to solve the ILP generated by any scheme implementation with GLPK
- the plots show different system configurations
    - a system configuration is given by "r x ( c \* i \* m )", which describes a system with *r* resources and *c* software components with *i* implementations having *m* modes each.
- for one such configuration the same experiment is run, i.e. the system is modified 7 times leading to the 7 steps, whereas only changes on hardware resources are made
    - for the cases involving Java (both, generation and solving), the same time is used for every step, as the generation always starts from skretch. Further, the changes (e.g. change the value of a single hardware resource while generation) can not be reflected by the Java System Generator
- on the x-axis, the steps of this manipulation are shown
    - the initial generation of the ILP (step zero) is only shown below in numbers, as it would skew the diagrams

Header = ['timestamp', 'dir', 'step', 'rows', 'cols', 'non-zero', 'ilp-sol', 'ti-ilp-sol']

In [83]:
racket_dats  = read_gen_results('plt-r6rs', since = datetime(2015,8,17,0,0,0))
larceny_dats = read_gen_results('larceny', since = datetime(2015,6,12,0,0,0))
java_dats    = read_gen_results('java')

Loaded 2124 records for plt-r6rs (8x32 unique) = 8 run(s)
Loaded 264 records for larceny (8x31 unique) = 1 run(s)
Loaded 88 records for java (1x31 unique) = 2 run(s)
Loaded 39 attribute metrics


In [12]:
java_glpk_dats = read_sol_results('java', since = datetime(2015,6,22,0,0,0))
glpk_dats      = read_sol_results('glpk')
gurobi_dats    = read_sol_results('gurobi')

Loaded 23 records for java (1x23 unique) = 1 run(s)
Loaded 1656 records for glpk (8x23 unique) = 9 run(s)
Loaded 216 records for gurobi (8x27 unique) = 1 run(s)


In [ ]:
def draw_gen(params):
    # needed number of axes equals ax_nr+1 now
    name = 'gen_{}'.format(changeName)
    f, ax_arr = plt.subplots(nrows = ax_nr+1, ncols = 3, sharex=True, sharey=True)
    f.set_size_inches(25.5,3.5*(ax_nr+1))
    one_plot = ax_arr.shape[1] == 1

    f.patch.set_facecolor('none')
    f.patch.set_alpha(0.0)

    lines, labels = [], []
    for p in params:
        ax_tup = ax_arr if one_plot else ax_arr[p[0]]
        ax_j = ax_tup[0]
        ax_r = ax_tup[1]
        ax_l = ax_tup[2]
    #    ax_j.set_ylim([0,50])
    #    ax_r.set_ylim([0,10])
    #    ax_l.set_ylim([0,10])
    #    x_g = np.array(xrange(1,len(p[1])+1)) # start at one, since first gen-time is cut
        x_g = np.array(xrange(START_STEP,len(p[1])+START_STEP)) # start at zero
        line_java    = ax_j.plot(x_g, p[8][0]*np.ones(len(p[1])), ls = p[5], c = p[6], label = p[7])
        line_racket  = ax_r.plot(x_g, p[1], ls = p[5], c = p[6], label = p[7])
        line_larceny = ax_l.plot(x_g, p[2], ls = p[5], c = p[6], label = p[7])
        ax_l.legend(loc='upper left', bbox_to_anchor=(1, 1.02))
        lines.append(line_racket[0])
        labels.append(p[7])

    for ax in ax_arr if one_plot else ax_arr.flatten():
        ax.set_ylabel('seconds')
        ax.patch.set_alpha(1)
        ax.patch.set_facecolor('white')

    first_ax = ax_arr if one_plot else ax_arr[0]
    plt.suptitle('ILP Generation Time', fontsize = 16)
    first_ax[0].set_title('Java')
    first_ax[1].set_title('Racket')
    first_ax[2].set_title('Larceny')

    # Fine-tune figure; make subplots close to each other and hide x ticks for
    # all but bottom plots.
    f.subplots_adjust(hspace=0.2)
    plt.setp([a.get_xticklabels() for a in f.axes[:-3]], visible=False)
    plt.setp([a.get_yticklabels() for a in f.axes], visible=True)
    plt.savefig('doc/{}.pdf'.format(name), facecolor=f.get_facecolor(), edgecolor='none')
    plt.savefig('{}.png'.format(name), facecolor=f.get_facecolor(), edgecolor='none')
    plt.close()

In [ ]:
def draw_sol(changeName, params):
    name = 'sol_{}'.format(changeName)
    f, ax_arr = plt.subplots(nrows = ax_nr+1, ncols = 3, sharex=True, sharey=True)
    f.set_size_inches(25.5,3.5*(ax_nr+1))
    one_plot = ax_arr.shape[1] == 1

    f.patch.set_facecolor('none')
    f.patch.set_alpha(0.0)

    lines, labels = [], []
    for p in params:
        ax_tup = ax_arr if one_plot else ax_arr[p[0]]
        ax_javaglpk = ax_tup[0]
        ax_glpk     = ax_tup[1]
        ax_gurobi   = ax_tup[2]
        x = np.array(xrange(0,len(p[3]))) # start at zero
        line_javaglpk = ax_javaglpk.plot(x, p[9][0]*np.ones(len(p[3])), ls = p[5], c = p[6], label = p[7])
        line_glpk     = ax_glpk.plot(x, p[3], ls = p[5], c = p[6], label = p[7])
        line_gurobi   = ax_gurobi.plot(x, p[4], ls = p[5], c = p[6], label = p[7])
        ax_gurobi.legend(loc='upper left', bbox_to_anchor=(1, 1.02))
        lines.append(line_glpk[0])
        labels.append(p[7])

    for ax in ax_arr if one_plot else ax_arr.flatten():
        ax.set_ylabel('seconds')
        ax.patch.set_alpha(1)
        ax.patch.set_facecolor('white')

    first_ax = ax_arr if one_plot else ax_arr[0]
    plt.suptitle('ILP Solving Time', fontsize = 16)
    first_ax[0].set_title('GLPK (Java)')
    first_ax[1].set_title('GLPK (Scheme)')
    first_ax[2].set_title('Gurobi (Scheme)')

    # Fine-tune figure; make subplots close to each other and hide x ticks for
    # all but bottom plots.
    f.subplots_adjust(hspace=0.2)
    plt.setp([a.get_xticklabels() for a in f.axes[:-3]], visible=False)
    plt.setp([a.get_yticklabels() for a in f.axes], visible=True)
    plt.savefig('doc/{}.pdf'.format(name))
    plt.savefig('{}.png'.format(name))
    plt.close()

In [ ]:
def draw_comp_sol(changeName, params):
    name = 'comp_sol_{}'.format(changeName)
    indices = [3, 12, 14, 19]
    for i in indices:
        p = params[i]
        line_javaglpk = plt.plot(x, p[9][0]*np.ones(len(p[3])), ls = linestyles[0], c = p[6], label = p[7])
        line_glpk     = plt.plot(x, p[3], ls = linestyles[1], c = p[6])
        line_gurobi   = plt.plot(x, p[4], ls = linestyles[3], c = p[6])
    plt.legend(loc = 'right')
    plt.ylabel('seconds')
    plt.suptitle('ILP Solving Time - Comparison', fontsize = 16)
    plt.savefig('doc/{}.pdf'.format(name))
    plt.savefig('{}.png'.format(name))
    plt.close()

In [13]:
for change_index in xrange(len(racket_dats) + 1):
    START_STEP = 0
    MAX_PLOTS_IN_ONE, current_plot, ax_nr, last_res = 7, 0, 0, -1
    params = []
    for i in xrange(len(specs)):
        current_res = specs[i][2]
        current_plot += 1
        if current_plot > MAX_PLOTS_IN_ONE and last_res != current_res:
            ax_nr += 1
            current_plot = 0
        params.append([ax_nr, safe(racket_dat[change_index],i,START_STEP), safe(larceny_dat[change_index],i,START_STEP),
                       safe(glpk_dat[change_index],i), safe(gurobi_dat[change_index],i),
                       line_def[i], color_def[i], '{2:d} x ({4}*{5}*{6})'.format(*specs[i]),
                       safe(java_dat[change_index],i), safe(java_glpk_dat[change_index],i)])
        last_res = current_res
    draw_gen(params)
    draw_sol(params)
    draw_comp_sol(params)

## Times for initial generation of the ILP

In [17]:
(racket_dat[0], larceny_dat[0])

(array([  0.82 ,   0.239,   2.284,  19.2  ,   0.21 ,   0.319,   3.414,
         27.39 ,   1.433,   2.482,   3.342,   6.396,  10.884,  12.314,
         14.289,  15.988,  21.376,  26.704,  32.28 ,  24.226,  29.95 ,
         36.352,  43.842,   3.219,   7.972,  12.263,  17.145,   0.352,
          0.763,   2.226,   9.606]),
 array([  0.17  ,   0.187 ,   1.766 ,  11.3   ,   0.3   ,   0.403 ,
          3.304 ,  22.73  ,   1.439 ,   2.375 ,   3.521 ,   6.308 ,
         10.359 ,  12.319 ,  15.182 ,  18.13  ,  22.5205,  27.992 ,
         32.143 ,  25.217 ,  30.606 ,  38.144 ,  45.88  ,   4.294 ,
          9.545 ,  16.402 ,  24.272 ,   0.362 ,   0.711 ,   1.518 ,   3.948 ]))

## Attribute metrics

- X = {normal, flushed, noncached}
- (X1,X2) = {(normal, flushed), (normal, noncached), (flushed, noncached)}

In [16]:
att_dat = read_att_result()
att_dat.dtype

Loaded 39 attribute metrics


dtype([('dir', '<f8'), ('normalBaseline', '<f8'), ('flushedBaseline', '<f8'), ('noncachedBaseline', '<f8'), ('ratioNormalToFlushed', '<f8'), ('ratioNormalToNoncached', '<f8'), ('ratioFlushedToNoncached', '<f8'), ('speedupNormalToFlushed', '<f8'), ('speedupNormalToNoncached', '<f8'), ('speedupFlushedToNoncached', '<f8')])

1) {X}basline: `total.X.computed / total.X.called`
- the baseline of the method X

In [12]:
width = 0.5
x = att_dat['dir']
for n, (l, c) in enumerate((('noncached', 'grey'), ('flushed', 'g'), ('normal', 'b'))):
    y = att_dat[l + 'Baseline']
    #plt.plot(x, y, label = l, color = c)
    plt.bar(x*4*width + n*width, y, width, label = l, color = c)

plt.legend(loc = 'best')
plt.ylabel('% computed')
plt.suptitle('Attribute Baselines', fontsize = 16)
plt.savefig('doc/att_bl.pdf')
!pdfcrop doc/att_bl.pdf doc/att_bl_cropped.pdf > /dev/null
plt.savefig('att_bl.png')
plt.close()

2) ratio{X1}To{X2}: `total.{X1}.computed / total.{X2}.called`
- the efficiency of the incremental approach in comparison to the method X2,
  i.e. the ratio between actual work done in X1 compared to possible work done with method X2

In [18]:
x = att_dat['dir']
for n, (name, c, ls) in enumerate((('FlushedToNoncached', 'orange', '--'),
                            ('NormalToFlushed', 'purple', '-'),
                            ('NormalToNoncached', 'r', '-.'))):
    y = att_dat['ratio' + name]
    plt.plot(x, y, label = name.replace('To', r'$\rightarrow$'), c = c, ls = ls)
    #plt.bar(x*3*width + n*width, y, width, label = l, color = c)

plt.legend(loc = 'best')
plt.ylabel('%')
plt.suptitle('Attribute Ratios', fontsize = 16)
plt.savefig('doc/att_r.pdf')
!pdfcrop doc/att_r.pdf doc/att_r_cropped.pdf > /dev/null
plt.savefig('att_r.png')
plt.close()

3) speedup{X1}To{X2}: `(total.{X2}.computed / total.{X2}.called) - (total.{X1}.computed / total.{X2}.called)`
- = `baseline({X2}) - ratio({X1}, {X2})`
- the "speed-up" of the incremental approach (normal or flushed) in comparison to the method X2

In [14]:
x = att_dat['dir']
for n, (l, c) in enumerate((('NormalToNoncached', 'r'),
                            ('FlushedToNoncached', 'orange'),
                            ('NormalToFlushed', 'yellow'))):
    y = att_dat['speedup' + l]
    plt.plot(x, y, label = l.replace('To', r'$\rightarrow$'), color = c)
    #plt.bar(x*3*width + n*width, y, width, label = l, color = c)

plt.legend(loc = 'best')
plt.ylabel('%')
plt.suptitle('Attribute Speed-Ups', fontsize = 16)
plt.savefig('doc/att_sp.pdf')
!pdfcrop doc/att_sp.pdf doc/att_sp_cropped.pdf > /dev/null
plt.savefig('att_sp.png')
plt.close()

## Boxplots for attribute measures

In [41]:
with open('profiling/kinds.json') as fd:
    d = json.load(fd)
att_totals = {}
for change in d['changes']:
    att_totals[change] = read_att_result(name = 'profiling/splitted/att-percentages_{}.csv'.format(change))

Loaded 31 attribute metrics
Loaded 4 attribute metrics
Loaded 4 attribute metrics


In [79]:
PRINT_NONCACHED = True
fig, axes = plt.subplots(ncols=len(d['changes']))
l = ['normal', 'flushed']
if PRINT_NONCACHED:
    l.append('noncached')
for i, change in enumerate(d['changes']):
    att_dat = att_totals[change]
    axes[i].boxplot([att_dat[name + 'Baseline'] for name in l], labels = [name[0]+name[3] for name in l])
    axes[i].set_title(change, fontsize=10)
    if i > 0:
        axes[i].set_yticklabels([])
if PRINT_NONCACHED:
    for ax in axes.flatten():
        ax.set_ylim([0,1.05])
plt.suptitle('Attribute Baselines', fontsize = 16)
plt.savefig('doc/att_box_bl.pdf')
!pdfcrop doc/att_box_bl.pdf doc/att_box_bl_cropped.pdf > /dev/null
plt.savefig('att_box_bl.png')
plt.close()